<a href="https://colab.research.google.com/github/Aayush-Jain01/Object-Detection/blob/main/Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ray[tune]

     |████████████████████████████████| 49.6 MB 6.2 kB/s 
     |████████████████████████████████| 72 kB 558 kB/s 
     |████████████████████████████████| 124 kB 43.9 MB/s 


In [ ]:
import torch
from torch import nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from torch.utils.data import random_split
import numpy as np
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
import os
import pandas as pd
from skimage import io, transform
from PIL import Image
import cv2
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.rpn import AnchorGenerator



In [ ]:
 from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
im_path = "/content/drive/MyDrive/YOLO Object Detection/training_images/vid_4_10000.jpg"
im = io.imread(im_path)
im.shape
im_ = im.transpose((2, 0, 1))

In [ ]:
class CarDataset(Dataset):
  def __init__(self, csv_file, root_dir, transform):
    super().__init__()
    self.csv_file = pd.read_csv(csv_file)
    self.root_dir = root_dir
    self.transform = transform
  def __len__(self):
    return len(self.csv_file)
  def __getitem__(self, idx):
    img_path = os.path.join(self.root_dir, self.csv_file.iloc[idx, 0])
    image = io.imread(img_path)
    image_ = Image.fromarray(image)
    bounding_params = self.csv_file.iloc[idx, 1:]
    #bounding_params = np.array([bounding_params])
    bounding_params = torch.as_tensor(bounding_params, dtype= torch.float32)
    labels = torch.ones((1,), dtype= torch.int64)
    image_id = torch.tensor([idx])
    area = ((bounding_params[3]-bounding_params[1])*(bounding_params[2]-bounding_params[0]))
    target = {}
    target["boxes"] = bounding_params
    target["labels"] = labels
    target["image_id"] = image_id
    target["area"] = area
    #sample = {'image': image, "bounding_parms" : bounding_params}
    if self.transform:
        image_1 = self.transform(image_)
    return image_1, target


In [ ]:
train_transforms = transforms.Compose([
        #transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        #transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(0.3, 0.3, 0.3, 0.3),
        transforms.RandomHorizontalFlip(),
        #transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
      ])
test_transforms = transforms.Compose([
        #transforms.Resize(size=256),
        #transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
train_set = CarDataset("/content/drive/MyDrive/YOLO Object Detection/train_solution_bounding_boxes (1).csv", "/content/drive/MyDrive/YOLO Object Detection/training_images/", transform= train_transforms)
train_set

In [ ]:
def collate_func(batch):
    return tuple(zip(*batch))
train_set, val_set = random_split(train_set, lengths=[447,112])
test_set = CarDataset("/content/drive/MyDrive/YOLO Object Detection/sample_submission.csv", "/content/drive/MyDrive/YOLO Object Detection/testing_images", test_transforms)
train_dl =  DataLoader(train_set, batch_size= 3, collate_fn= collate_func, shuffle = False)
val_dl = DataLoader(val_set, batch_size= 4, collate_fn= collate_func, shuffle = True)
test_dl = DataLoader(test_set, batch_size = 4, collate_fn = collate_func, shuffle = False)


In [ ]:
def show_image(image, bounding_params):
  image_ = image.transpose(*(0, 2))
  image_2 = image_.transpose(*(1, 0))
  image_3 = image_2.numpy()
  image_4 = cv2.cvtColor(image_3, cv2.COLOR_BGR2RGB)
  plt.imshow(image_4)
  print(bounding_params)


In [ ]:
from torchvision import models


In [ ]:
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
for parameters in model.parameters():
  parameters.requires_grad = False
num_classes = 2 
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

In [ ]:
model

In [ ]:
model.to('cuda')

In [ ]:
import sklearn.metrics
def precision_recall_curve(y_true, pred_scores, thresholds):
    precisions = []
    recalls = []
    for threshold in thresholds:
        y_pred = [1 if score >= threshold else 0 for score in pred_scores]
        
        precision = sklearn.metrics.precision_score(y_true=y_true, y_pred=y_pred, pos_label=1)
        recall = sklearn.metrics.recall_score(y_true=y_true, y_pred=y_pred, pos_label=1)
        
        precisions.append(precision)
        recalls.append(recall)

    return precisions, recalls
def predictions(preds, images, targets):
    #pred_labels = pred["labels"]
    scores = []
    true_labels = []
    #pred_labels = [1 if score >= threshold else 0 for score in scores] 
    for i in range(len(targets)):
      target = targets[i]
      true_labels.append(target["labels"])
    thresholds = np.arange(0.2, 0.9, step= 0.25)
    APS = []
    for i in range(len(targets)):
      target = targets[i]
      true_labels = target["labels"]
      precisions, recalls = precision_recall_curve(y_true=true_labels, pred_scores=preds[i]["scores"], thresholds=thresholds)
      precisions.append(1)
      recalls.append(0)

      precisions = numpy.array(precisions)
      recalls = numpy.array(recalls)

      AP = numpy.sum((recalls[:-1] - recalls[1:]) * precisions[:-1])
      APS.append(AP)
    print("Mean Average Precision = " + str(sum(APS)/4))

In [ ]:
def train(config): 
  optimizer = torch.optim.SGD(model.parameters(), lr= config["lr"], weight_decay= config["weight_decay"])
  lr_schedular = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr= config["base_lr"], max_lr = config["max_lr"], gamma= config["gamma"]) 
  for epoch in range(10):
    model.train()
    k = 1
    for images, targets in train_dl:
      print(k)
      k += 1
      optimizer.zero_grad()
      loss_clas = []
      loss_regs = []
      # target_new = {}
      # boxes = []
      # labels = []
      # image_ids = []
      # areas = []
      # for i in range(len(targets)):
      #   target = targets[i]
      #   boxes.append(target["boxes"])
      #   labels.append(target["labels"])
      #   image_ids.append(target["image_id"])
      #   areas.append(target["area"])
      # target_new["boxes"] = torch.stack(boxes)
      # target_new["labels"] = torch.stack(labels)
      # target_new["image_id"] = torch.stack(image_ids)
      # target_new["area"] = torch.stack(areas)
      for i in range(len(targets)):
        targets[i]["boxes"] = torch.reshape(targets[i]["boxes"], (1, 4))
      images_ = list(image.to('cuda') for image in images)
      targets_ = [{k: v.to('cuda') for k, v in t.items()} for t in targets]
      loss = model(images_, targets_)
      loss_cla = loss['loss_classifier']
      loss_reg = loss['loss_box_reg']
      loss_clas.append(loss_cla)
      loss_regs.append(loss_reg)
      loss_sum = sum(loss for loss in loss.values())
      loss_sum.backward()
      optimizer.step()
    lr_schedular.step()
    print("Average Total Loss = " + str(loss_sum/4) + " in epoch = " + str(epoch))
    print("Average CLA Loss = " + str(sum(loss_clas)/len(loss_clas)) + " Average REG Loss = " + str(sum(loss_regs)/len(loss_regs)) + "Epoch =" + str(epoch))
    model.eval()
    with torch.no_grad():
      for images, targets in val_dl:
        # target_new = {}
        # boxes = []
        # labels = []
        # image_ids = []
        # areas = []
        # for i in range(len(targets)):
        #   target = targets[i]
        #   boxes.append(target["boxes"])
        #   labels.append(target["labels"])
        #   image_ids.append(target["image_id"])
        #   areas.append(target["area"])
        # target_new["boxes"] = torch.stack(boxes)
        # target_new["labels"] = torch.stack(labels)
        # target_new["image_id"] = torch.stack(image_ids)
        # target_new["area"] = torch.stack(areas)
        for i in range(len(targets)):
          targets[i]["boxes"] = torch.reshape(targets[i]["boxes"], (1, 4))
        images_ = list(image.to('cuda') for image in images)
        targets_ = [{k: v.to('cuda') for k, v in t.items()} for t in targets]
        preds = model(images_, targets_)
        # print(targets_)
        # print(preds)
        predictions(preds, images_, targets_)
    

In [ ]:
config = {
    "lr" : 1e-4,
    "weight_decay" : 1e-5,
    "base_lr" : 1e-5,
    "max_lr" : 5e-3,
    "gamma" : 1.0
}
train(config)